## Iris Dataset classification prediction using PyTorch


## Reference:
Thanks for excellent tutorial for Pytorch!
https://www.youtube.com/watch?v=lfQs6JEkhTU

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


In [2]:
from sklearn import datasets

In [3]:
SEED = 1239
torch.manual_seed(SEED)


In [4]:
if torch.backends.mps.is_available():
    torch.cuda.manual_seed(SEED)  #will it work?

    
np.random.seed(SEED)

In [5]:
from sklearn.datasets import load_iris

iris = load_iris()

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= ['slength','swidth','plength','pwidth'] + ['class'])


In [6]:
df = df.astype(np.float32)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   slength  150 non-null    float32
 1   swidth   150 non-null    float32
 2   plength  150 non-null    float32
 3   pwidth   150 non-null    float32
 4   class    150 non-null    float32
dtypes: float32(5)
memory usage: 3.1 KB


In [8]:
n = len(df.index)
new_indices = np.random.permutation(n)
df = df.iloc[new_indices]

In [9]:
y = df ['class']
X = df.drop('class', axis=1)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X,y)


In [11]:
X_scaled[:10]

array([[ 1.1591729e+00, -1.3197948e-01,  9.9010795e-01,  1.1855671e+00],
       [ 1.6438439e+00,  3.2841417e-01,  1.2742952e+00,  7.9067063e-01],
       [-1.7367418e-01, -1.2829634e+00,  7.0592082e-01,  1.0539351e+00],
       [-4.1600969e-01, -1.0527668e+00,  3.6489633e-01,  8.7762193e-04],
       [-1.1430168e+00,  1.2492008e+00, -1.3402265e+00, -1.4470764e+00],
       [-1.1430168e+00,  9.8217063e-02, -1.2833891e+00, -1.3154444e+00],
       [-2.9484195e-01, -3.6217603e-01, -8.9803182e-02,  1.3250968e-01],
       [ 3.1099743e-01, -5.9237313e-01,  5.3540844e-01,  8.7762193e-04],
       [ 2.4920194e+00,  1.7095945e+00,  1.5016450e+00,  1.0539351e+00],
       [ 6.8661921e-02,  3.2841417e-01,  5.9224612e-01,  7.9067063e-01]],
      dtype=float32)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(),
                                                    stratify=y, 
                                                    test_size=.4)

In [13]:
X_train.shape, y_train.shape
x_shape = X_train.shape
y_shape = y_train.shape



In [14]:
input_size = x_shape[1]
output_size = len(np.unique(y_train))

In [15]:
print(input_size,output_size)

4 3


## Dataset Class

The dataset class defines a class where data exists. It has two types, test dataset and train dataset. Each type will hold the corresponding data.

In [16]:
class IrisDataset(Dataset):
    def __init__(self, data,label):
        assert len(data) == len(label)
        
        self.data = torch.from_numpy(data)
        self.label =torch.from_numpy(label)
        
    def __getitem__ (self,index):
        return self.data[index], self.label[index]
    
    def __len__(self):
        return len(self.label)
    



#### Load data to dataloader dataset

In [17]:
train_data = IrisDataset(X_train,y_train)
test_data = IrisDataset(X_test,y_test)


train_dataloader = DataLoader(
    train_data,
    batch_size = 64,
    shuffle = False
)

test_dataloader = DataLoader(
    train_data,
    batch_size = 64,
    shuffle = False
)

### Create neural network

#### Attach to MPS (Metal), else CPU

In [18]:
device = torch.device('cpu' if torch.backends.mps.is_available() else 'cpu')

In [19]:
device

device(type='cpu')

In [20]:
class IrisNeuralNetwork(nn.Module):
    def __init__(self):
        super(IrisNeuralNetwork, self).__init__()
        
        self.fn1 = nn.Linear(input_size,6)
        self.fn2 = nn.Linear(6,output_size)
        
    def forward(self,x):
        x = F.relu(self.fn1(x))
        x = self.fn2(x)
        return x
        

In [21]:
model = IrisNeuralNetwork()
model.to(device)

IrisNeuralNetwork(
  (fn1): Linear(in_features=4, out_features=6, bias=True)
  (fn2): Linear(in_features=6, out_features=3, bias=True)
)

In [22]:
x,y = next(iter(train_dataloader))
x = x[:5].to(device)
score = model(x)
print(score)

tensor([[ 1.9031,  0.1955,  1.9378],
        [ 1.7876,  0.2963,  1.9375],
        [ 1.1618,  0.2885,  2.1160],
        [ 1.6410,  0.2266,  1.9201],
        [ 1.6470, -0.0170,  1.9157]], grad_fn=<AddmmBackward0>)


## Loss Function and Optimizer

Cross Entropy Loss and Adam Optimizer


In [23]:

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.01)


In [24]:
def train():
    model.train()
    
    for x,y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        n = x.size(0)
        
        optimizer.zero_grad()
        score = model(x)
        y_long = y.type(torch.LongTensor)
        loss = loss_function(score,y_long)
        
        loss.backward()
        optimizer.step()
        
        predictions = score.max(1, keepdim=True)[1]
        num_correct = predictions.eq(y.view_as(predictions)).sum().item()
        
    accuracy = num_correct /n
    return loss, accuracy
        

In [25]:
def evaluate():
    model.eval()
    
    with torch.no_grad():
        for x,y in test_dataloader:
            x = x.to(device)
            y = y.to(device)
            n = x.size(0)

            score = model(x)
            y_long = y.type(torch.LongTensor)
            loss = loss_function(score,y_long)

            predictions = score.max(1, keepdim=True)[1]
            num_correct = predictions.eq(y.view_as(predictions)).sum().item()
        
    accuracy = num_correct /n 
    return loss, accuracy

In [26]:
max_epochs = 80
for epoch in range(max_epochs):
    tr_loss, tr_acc = train()
    eva_loss, eva_acc = evaluate()
    
        
    print('[{}/{}] Train loss:{:.4f} acc:{:.2f}% -- Test Loss:{:.4f} acc:{:.2f}%'.format(
        epoch, max_epochs, tr_loss,tr_acc*100, eva_loss, eva_acc*100
    ))
    
    if eva_acc >= 1:
        break

[0/80] Train loss:1.3220 acc:3.85% -- Test Loss:1.2136 acc:26.92%
[1/80] Train loss:1.1661 acc:46.15% -- Test Loss:1.1273 acc:38.46%
[2/80] Train loss:1.1361 acc:15.38% -- Test Loss:1.1329 acc:26.92%
[3/80] Train loss:1.1405 acc:26.92% -- Test Loss:1.1172 acc:26.92%
[4/80] Train loss:1.0984 acc:23.08% -- Test Loss:1.0635 acc:15.38%
[5/80] Train loss:1.0407 acc:26.92% -- Test Loss:1.0143 acc:57.69%
[6/80] Train loss:1.0012 acc:57.69% -- Test Loss:0.9879 acc:57.69%
[7/80] Train loss:0.9836 acc:57.69% -- Test Loss:0.9775 acc:57.69%
[8/80] Train loss:0.9766 acc:57.69% -- Test Loss:0.9722 acc:57.69%
[9/80] Train loss:0.9720 acc:57.69% -- Test Loss:0.9675 acc:57.69%
[10/80] Train loss:0.9681 acc:57.69% -- Test Loss:0.9639 acc:57.69%
[11/80] Train loss:0.9659 acc:57.69% -- Test Loss:0.9616 acc:57.69%
[12/80] Train loss:0.9637 acc:57.69% -- Test Loss:0.9573 acc:53.85%
[13/80] Train loss:0.9566 acc:42.31% -- Test Loss:0.9458 acc:50.00%
[14/80] Train loss:0.9405 acc:42.31% -- Test Loss:0.9252 ac